## L3 Order Book - Implementation

---

## Introduction

An order book is the core data structure of modern electronic exchanges, maintaining all outstanding limit orders - instructions to buy (bid) or sell (ask) at specific prices. Moreover they are used to handle market orders - instructions to execute a bid or ask at the best ask or best bid respectively. Incoming orders are continuously matched against resting orders using strict price-time priority.

Vendors provide order book data at three levels of detail:

* L1: Best bid/ask prices + sizes only  
* L2: Full depth across all price levels (aggregated quantities)  
* L3: Individual orders per level

An example shows what an L3 order book might look like: 
```python
NOTATION: PRICE | [ID:QUANTITY_i]

BIDS                               ASKS
99.90 | [ID-459:100, 50]    100.10 | [ID-600:150, ID-459:50]
99.80 | [ID-459:300]        100.20 | [ID-600:100]
                                    
Spread = 0.20
```

## Goal

We aim to implement an minimally algorithmically efficient L3 Order Book, with:

Complexity:
* Insertion: $O(\log n)$ insertion on first order at limit, $O(1)$ otherwise
* Cancellation: $O(1)$
* Execute: $O(1)$
* Get volume at price $x$: $O(1)$
* Get best bid/offer: $O(1)$

To do this 

We don't additionally consider latency or thread/process or async usage, this will be deferred to the Order Book Service notebook.

## Implementation

In [1]:
import logging
import time
from dataclasses import dataclass
from datetime import datetime
from decimal import Decimal
from enum import Enum
from typing import Any, Callable, Dict, List, Optional, Tuple

from sortedcontainers import SortedDict

from theoria.validor import TestCase, Validor

In [2]:
log = logging.getLogger(__name__)
log.setLevel(logging.INFO)
if not log.hasHandlers():
    handler = logging.StreamHandler()
    handler.setFormatter(logging.Formatter("[%(asctime)s] [%(levelname)s] %(message)s"))
    log.addHandler(handler)

In [3]:
class Side(Enum):
    BID = "BID"
    ASK = "ASK"


@dataclass
class Order:
    """Represents a limit order in the order book."""

    order_id: int
    participant_id: str
    side: Side
    price: Decimal  # Use Decimal for precise floating-point arithmetic
    qty: Decimal
    timestamp: float  # Unix timestamp

    def __repr__(self):
        return f"Order(id={self.order_id}, participant_id={self.participant_id}, side={self.side.name}, price={self.price}, qty={self.qty}, time={datetime.fromtimestamp(self.timestamp)})"


class Level:
    """
    Price level with price-time priority using doubly-linked list.
    O(1) insert, O(1) cancel, O(1) FIFO head access.
    """

    class _OrderNode:
        """Doubly-linked list node for perfect FIFO at price level."""

        def __init__(self, order: Order):
            self.order = order
            self.prev: Optional["Level._OrderNode"] = None
            self.next: Optional["Level._OrderNode"] = None

    def __init__(self) -> None:
        self.total_qty: Decimal = Decimal(0)
        self.head: Optional[Level._OrderNode] = None
        self.tail: Optional[Level._OrderNode] = None
        self.order_to_node: Dict[int, Level._OrderNode] = {}
        self.order_count: int = 0

    def add_order(self, order: Order) -> None:
        """O(1) append to tail - maintains arrival order."""
        node = Level._OrderNode(order)
        self.order_to_node[order.order_id] = node

        if self.tail:
            # Append after tail
            self.tail.next = node
            node.prev = self.tail
            self.tail = node
        else:
            # First order
            self.head = self.tail = node

        self.total_qty += order.qty
        self.order_count += 1

    def remove_order(self, order_id: int) -> None:
        """O(1) removal - unlink from anywhere in list."""
        node = self.order_to_node.pop(order_id, None)
        if not node:
            return

        # Unlink from doubly-linked list (works for head/tail/middle)
        if node.prev:
            node.prev.next = node.next
        else:
            self.head = node.next

        if node.next:
            node.next.prev = node.prev
        else:
            self.tail = node.prev

        self.total_qty -= node.order.qty
        self.order_count -= 1

    def get_head(self) -> Optional[Order]:
        """O(1) FIFO head for matching."""
        return self.head.order if self.head else None

    def pop_head(self) -> Optional[Order]:
        """O(1) remove and return FIFO head."""
        node = self.head
        if not node:
            return None

        order = node.order
        self.remove_order(order.order_id)
        return order


class SideBook:
    """Order book for one side (bids or asks)."""

    def __init__(self, side: Side):
        self.side = side
        self.levels: SortedDict[Decimal, Level] = SortedDict()

    def best_price(self) -> Optional[Decimal]:
        """O(1) best price lookup."""
        if not self.levels:
            return None
        return (
            self.levels.peekitem(-1)[0]  # Highest bid
            if self.side is Side.BID
            else self.levels.peekitem(0)[0]  # Lowest ask
        )


class OrderBookError(Exception):
    pass

In [4]:
class OrderBook:
    def __init__(
        self,
        # --- Event Callbacks ---
        on_insert: Optional[Callable[[Order], Any]] = None,
        on_cancel: Optional[Callable[[Order], Any]] = None,
        on_fill: Optional[Callable[[int, Decimal, Decimal, float], Any]] = None,
    ):
        self.bids = SideBook(Side.BID)
        self.asks = SideBook(Side.ASK)

        self.orders_by_id: Dict[int, Order] = {}

        self.on_insert = on_insert
        self.on_cancel = on_cancel
        self.on_fill = on_fill

    def _side_book(self, side: Side) -> SideBook:
        return self.bids if side is Side.BID else self.asks

    def _remove_from_level(self, order: Order) -> None:
        """O(1) removal using doubly-linked list."""
        sb = self._side_book(order.side)
        if order.price not in sb.levels:
            return
        level = sb.levels[order.price]
        level.remove_order(order.order_id)

        if level.total_qty <= 0 or level.order_count == 0:
            del sb.levels[order.price]

    def _match_against(
        self,
        taker_side: Side,
        limit_price: Decimal,
        qty: Decimal,
        timestamp: float,
    ) -> Decimal:
        """
        FIFO matching engine:
        - Matches taker order against opposite side until limit price or quantity exhausted.
        - Executes trades at maker prices with price-time priority.
        - Calls on_fill callback for each executed trade.
        - Returns remaining quantity to be added to book (if any).
        """
        if qty <= 0:
            raise OrderBookError("Quantity must be positive")

        opposite_book = self._side_book(
            Side.ASK if taker_side is Side.BID else Side.BID
        )

        def price_crosses(best_price: Decimal) -> bool:
            return (
                best_price <= limit_price
                if taker_side is Side.BID
                else best_price >= limit_price
            )

        while qty > 0:
            best_price = opposite_book.best_price()
            if best_price is None or not price_crosses(best_price):
                break

            # Match against FIFO head at best price level
            level = opposite_book.levels[best_price]
            maker = level.get_head()
            if maker is None:
                del opposite_book.levels[best_price]
                continue

            # Execute trade - partial or full fill
            trade_qty = min(qty, maker.qty)
            maker.qty -= trade_qty
            level.total_qty -= trade_qty
            qty -= trade_qty

            if self.on_fill:
                self.on_fill(maker.order_id, maker.price, trade_qty, timestamp)

            if maker.qty == 0:
                # O(1) FIFO
                level.pop_head()
                self.orders_by_id.pop(maker.order_id, None)

        return qty

    # --- Public API ---

    def insert(
        self,
        order_id: int,
        participant_id: str,
        side: Side,
        price: Decimal,
        qty: Decimal,
        timestamp: Optional[float] = None,
    ) -> None:
        """Insert limit order with price-time priority."""
        if order_id in self.orders_by_id:
            raise OrderBookError(f"Order ID {order_id} already exists.")

        ts = timestamp if timestamp is not None else time.time()
        remaining = self._match_against(side, price, qty, ts)

        if remaining > 0:
            sb = self._side_book(side)
            if price not in sb.levels:
                sb.levels[price] = Level()

            level = sb.levels[price]
            order = Order(order_id, participant_id, side, price, remaining, ts)
            level.add_order(order)
            self.orders_by_id[order_id] = order

            if self.on_insert:
                self.on_insert(order)

    def cancel(self, order_id: int) -> bool:
        """O(1) cancel with price-time priority preservation."""
        order = self.orders_by_id.pop(order_id, None)
        if not order:
            return False

        self._remove_from_level(order)
        if self.on_cancel:
            self.on_cancel(order)
        return True

    def get_volume_at_price(self, side: Side, price: Decimal) -> Decimal:
        """O(1) volume at price."""
        sb = self._side_book(side)
        level = sb.levels.get(price)
        return level.total_qty if level else Decimal(0)

    def get_volume(self, price: Decimal) -> Dict[Side, Decimal]:
        """O(1) volume both sides."""
        return {
            Side.BID: self.get_volume_at_price(Side.BID, price),
            Side.ASK: self.get_volume_at_price(Side.ASK, price),
        }

    def best_bid(self) -> Optional[Tuple[Decimal, Decimal]]:
        """O(1) best bid."""
        price = self.bids.best_price()
        if price is None:
            return None
        return price, int(self.bids.levels[price].total_qty)

    def best_ask(self) -> Optional[Tuple[Decimal, Decimal]]:
        """O(1) best ask."""
        price = self.asks.best_price()
        if price is None:
            return None
        return price, int(self.asks.levels[price].total_qty)

    def get_orders_at_price(self, price: Decimal) -> Dict[Side, List[Order]]:
        """O(n) traversal - returns current orders in FIFO order."""
        result = {}
        for side, sb in [(Side.BID, self.bids), (Side.ASK, self.asks)]:
            level = sb.levels.get(price)
            if level and level.head:
                orders = []
                node = level.head
                while node:
                    orders.append(node.order)
                    node = node.next
                result[side] = orders
            else:
                result[side] = []
        return result

    def orders(self) -> Dict[Side, Dict[Decimal, List[Order]]]:
        """
        O(n * m) traversal - returns current orders in FIFO order.
        n = number of price levels, m = number of orders per level.
        """
        result = {}
        for side, sb in [(Side.BID, self.bids), (Side.ASK, self.asks)]:
            result[side] = {}
            for price, level in sb.levels.items():
                if level and level.head:
                    orders = []
                    node = level.head
                    while node:
                        orders.append(node.order)
                        node = node.next
                    result[side][price] = orders
                else:
                    result[side][price] = []
        return result

## Tests

In [5]:
test_cases = [
    TestCase(
        input_data={
            "operations": [
                (
                    "insert",
                    {
                        "order_id": 1,
                        "participant_id": "A",
                        "side": Side.BID,
                        "price": Decimal(100.0),
                        "qty": Decimal(10),
                    },
                ),
                (
                    "insert",
                    {
                        "order_id": 2,
                        "participant_id": "B",
                        "side": Side.ASK,
                        "price": Decimal(101.0),
                        "qty": Decimal(5),
                    },
                ),
                (
                    "insert",
                    {
                        "order_id": 3,
                        "participant_id": "C",
                        "side": Side.BID,
                        "price": Decimal(100.0),
                        "qty": Decimal(15),
                    },
                ),
                (
                    "insert",
                    {
                        "order_id": 4,
                        "participant_id": "D",
                        "side": Side.ASK,
                        "price": Decimal(100.5),
                        "qty": Decimal(10),
                    },
                ),
            ],
        },
        expected_output={
            "best_bid": (Decimal(100.0), Decimal(25)),
            "best_ask": (Decimal(100.5), Decimal(10)),
        },
        description="Basic insertions without matching",
    ),
    TestCase(
        input_data={
            "operations": [
                (
                    "insert",
                    {
                        "order_id": 1,
                        "participant_id": "A",
                        "side": Side.BID,
                        "price": Decimal(99.0),
                        "qty": Decimal(20),
                    },
                ),
                (
                    "insert",
                    {
                        "order_id": 2,
                        "participant_id": "B",
                        "side": Side.ASK,
                        "price": Decimal(100.0),
                        "qty": Decimal(10),
                    },
                ),
                # Matches
                (
                    "insert",
                    {
                        "order_id": 3,
                        "participant_id": "C",
                        "side": Side.BID,
                        "price": Decimal(100.0),
                        "qty": Decimal(5),
                    },
                ),
            ],
        },
        expected_output={
            "best_bid": (Decimal(99.0), Decimal(20)),
            "best_ask": (Decimal(100.0), Decimal(5)),
        },
        description="Insertion with matching against existing orders",
    ),
    TestCase(
        input_data={
            "operations": [
                (
                    "insert",
                    {
                        "order_id": 1,
                        "participant_id": "A",
                        "side": Side.BID,
                        "price": Decimal(97.0),
                        "qty": Decimal(10),
                    },
                ),
                (
                    "insert",
                    {
                        "order_id": 2,
                        "participant_id": "B",
                        "side": Side.ASK,
                        "price": Decimal(98.0),
                        "qty": Decimal(10),
                    },
                ),
                # Matches
                (
                    "insert",
                    {
                        "order_id": 3,
                        "participant_id": "C",
                        "side": Side.BID,
                        "price": Decimal(99.0),
                        "qty": Decimal(20),
                    },
                ),
                # Matches
                (
                    "insert",
                    {
                        "order_id": 4,
                        "participant_id": "A",
                        "side": Side.ASK,
                        "price": Decimal(96.0),
                        "qty": Decimal(5),
                    },
                ),
            ],
        },
        expected_output={
            "best_bid": (99.0, 5),
            "best_ask": None,
        },
        description="Multiple matching orders, matching via opposite price",
    ),
    TestCase(
        input_data={
            "operations": [
                (
                    "insert",
                    {
                        "order_id": 1,
                        "participant_id": "A",
                        "side": Side.BID,
                        "price": 98.0,
                        "qty": 30,
                    },
                ),
                ("cancel", {"order_id": 1}),
                (
                    "insert",
                    {
                        "order_id": 2,
                        "participant_id": "B",
                        "side": Side.ASK,
                        "price": 99.0,
                        "qty": 15,
                    },
                ),
            ],
        },
        expected_output={
            "best_bid": None,
            "best_ask": (99.0, 15),
        },
        description="Cancellation of an existing order",
    ),
    TestCase(
        input_data={
            "operations": [
                (
                    "insert",
                    {
                        "order_id": 1,
                        "participant_id": "A",
                        "side": Side.BID,
                        "price": 100.0,
                        "qty": 10,
                    },
                ),
                (
                    "insert",
                    {
                        "order_id": 2,
                        "participant_id": "B",
                        "side": Side.BID,
                        "price": 100.0,
                        "qty": 5,
                    },
                ),
                (
                    "insert",
                    {
                        "order_id": 3,
                        "participant_id": "C",
                        "side": Side.ASK,
                        "price": 101.0,
                        "qty": 20,
                    },
                ),
                (
                    "insert",
                    {
                        "order_id": 4,
                        "participant_id": "A",
                        "side": Side.ASK,
                        "price": 102.0,
                        "qty": 15,
                    },
                ),
            ],
        },
        expected_output={
            "get_volume": {
                100.0: {Side.BID: 15, Side.ASK: 0},
                101.0: {Side.BID: 0, Side.ASK: 20},
                102.0: {Side.BID: 0, Side.ASK: 15},
            },
        },
        description="Volume lookup on both sides at specific price levels",
    ),
    TestCase(
        input_data={
            "operations": [
                (
                    "insert",
                    {
                        "order_id": 1,
                        "participant_id": "A",
                        "side": Side.BID,
                        "price": Decimal(100.0),
                        "qty": Decimal(10),
                        "timestamp": 0,
                    },
                ),
                (
                    "insert",
                    {
                        "order_id": 2,
                        "participant_id": "B",
                        "side": Side.BID,
                        "price": Decimal(100.0),
                        "qty": Decimal(5),
                        "timestamp": 1,
                    },
                ),
                (
                    "insert",
                    {
                        "order_id": 3,
                        "participant_id": "C",
                        "side": Side.ASK,
                        "price": Decimal(101.0),
                        "qty": Decimal(20),
                        "timestamp": 2,
                    },
                ),
                (
                    "insert",
                    {
                        "order_id": 4,
                        "participant_id": "D",
                        "side": Side.ASK,
                        "price": Decimal(102.0),
                        "qty": Decimal(15),
                        "timestamp": 3,
                    },
                ),
            ],
        },
        expected_output={
            "get_orders_at_price_100.0": {
                Side.BID: [
                    Order(1, "A", Side.BID, Decimal(100.0), Decimal(10), 0),
                    Order(2, "B", Side.BID, Decimal(100.0), Decimal(5), 1),
                ],
                Side.ASK: [],
            },
        },
        description="Retrieve orders at specific price level in FIFO order",
    ),
]


def run_lob_test(operations: List[Tuple[str, Dict]]) -> OrderBook:
    book = OrderBook()

    for op_name, params in operations:
        fn = getattr(book, op_name)
        fn(**params)

    return book


def compare_results(actual: OrderBook, expected: Dict[str, Any]) -> bool:
    if "best_bid" in expected:
        if actual.best_bid() != expected["best_bid"]:
            return False
    if "best_ask" in expected:
        if actual.best_ask() != expected["best_ask"]:
            return False
    if "get_volume" in expected:
        for price, vols in expected["get_volume"].items():
            actual_vols = actual.get_volume(price)
            for side, vol in vols.items():
                if actual_vols.get(side, 0) != vol:
                    return False
    if "get_orders_at_price" in expected:
        for price_key, orders_dict in expected["get_orders_at_price"].items():
            price = Decimal(price_key.split("_")[-1])
            actual_orders = actual.get_orders_at_price(price)
            for side, expected_orders in orders_dict.items():
                actual_side_orders = actual_orders.get(side, [])
                if len(actual_side_orders) != len(expected_orders):
                    return False
                for act_order, exp_order in zip(
                    actual_side_orders, expected_orders, strict=True
                ):
                    if (
                        act_order.order_id != exp_order.order_id
                        or act_order.participant_id != exp_order.participant_id
                        or act_order.side != exp_order.side
                        or act_order.price != exp_order.price
                        or act_order.qty != exp_order.qty
                        or act_order.timestamp != exp_order.timestamp
                    ):
                        return False
    return True


if __name__ == "__main__":
    Validor(run_lob_test).add_cases(test_cases).run(compare_results)

[2026-01-16 12:27:49,473] [INFO] All 6 tests passed for run_lob_test.


## References

https://github.com/Kautenja/limit-order-book/blob/master/notes/lob.md